<a href="https://colab.research.google.com/github/CallMeIsEthan/TriTueNhanTao/blob/main/Tuan03_Minimax_AlphaBeta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font face="Times New Roman" size="3">**BÁO CÁO MÔN HỌC THỰC HÀNH TRÍ TUỆ NHÂN TẠO**<br>
**Chủ đề:** Thuật toán tìm kiếm đối kháng<br>
**Sinh viên thực hiện:** Phan Tuấn Giang - 2001230198<br>
</font>

<font face="Times New Roman">

**CHƯƠNG 1: TỔNG QUAN**

1.1. Trí tuệ nhân tạo trong game

<div align= "justify">

&emsp;&emsp;Trí tuệ nhân tạo (Artificial Intelligence – AI) trong game là lĩnh vực nghiên cứu và phát triển các thuật toán giúp máy tính có khả năng chơi game một cách thông minh, đưa ra quyết định giống hoặc vượt trội con người. Trong các trò chơi hai người luân phiên (như cờ caro, cờ vua, cờ vây, tic-tac-toe,...), AI thường được xây dựng dưới dạng một tác tử (agent) có khả năng:

&emsp;&emsp;&emsp;&emsp;- Phân tích trạng thái bàn chơi hiện tại.

&emsp;&emsp;&emsp;&emsp;- Dự đoán các nước đi tiếp theo của cả hai bên.

&emsp;&emsp;&emsp;&emsp;- Lựa chọn nước đi tối ưu nhằm đạt được thắng lợi hoặc ít nhất là không thua.

&emsp;&emsp;Việc ứng dụng AI trong game không chỉ dừng lại ở mục đích giải trí, mà còn là một môi trường lý tưởng để nghiên cứu các thuật toán tìm kiếm, ra quyết định, tối ưu hóa và lý thuyết trò chơi. Một trò chơi bảng đơn giản cũng có thể ẩn chứa không gian trạng thái rất lớn, là bài toán thực tế để kiểm chứng hiệu quả của các thuật toán tìm kiếm đối kháng.

&emsp;&emsp;Trong bối cảnh đó, các thuật toán như Minimax và Cắt tỉa Alpha-Beta đóng vai trò trung tâm. Chúng là nền tảng cho khả năng “suy nghĩ nhiều bước” của máy, giúp hệ thống có thể dự đoán phản ứng của đối thủ và đưa ra chiến lược phù hợp. Vì vậy, việc nghiên cứu các thuật toán này mang ý nghĩa quan trọng trong cả học thuật lẫn ứng dụng.

</div>

1.2. Trò chơi tổng bằng không (Zero-sum Game)  

1.3. Các thành phần của mô hình trò chơi  

1.4. Mục tiêu và phạm vi  

1.5. Phân tích không gian trạng thái  



CHƯƠNG 2: CƠ SỞ LÝ THUYẾT  

2.1. Cây trò chơi (Game Tree)  

2.2. Thuật toán Minimax  

2.3. Cắt tỉa Alpha-Beta  

2.4. Hiệu ứng đường chân trời (Horizon Effect)  

2.5. Vai trò của heuristic  

2.6. Các kỹ thuật hỗ trợ nâng cao  



CHƯƠNG 3: PHÂN TÍCH VÀ THIẾT KẾ CHƯƠNG TRÌNH  

3.1. Phát biểu bài toán  

3.2. Thiết kế cấu trúc dữ liệu  

3.3. Đầu vào – đầu ra thuật toán  

3.4. Thiết kế hàm đánh giá (Heuristic Function)  

3.5. Mã giả (Pseudocode)  



CHƯƠNG 4: THỰC NGHIỆM VÀ ĐÁNH GIÁ  

4.1. Môi trường cài đặt  

4.2. Kịch bản thử nghiệm  

4.3. Kết quả  

4.4. Phân tích  

</font>

In [ ]:
import copy
import math
import random
import numpy

In [ ]:
X = "X"
O = "O"
EMPTY = None
user = None
ai = None

In [ ]:
def initial_state():
  return [[EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY]]

In [ ]:
def player(board):
  count = 0
  for i in board:
    for j in i:
      if j:
        count += i
  if count % 2 != 0:
    return ai
  return user

In [ ]:
def actions(board):
  res = set()
  board_len = len(board)
  for i in range(board_len):
    for j in range(board_len):
      if board[i][j] == EMPTY:
        res.add((i, j))
  return res

In [ ]:
def get_horizontal_winner(board):
    """
    Kiểm tra thắng theo hàng.
    Trả về X hoặc O nếu có hàng thắng, ngược lại trả về None.
    """
    board_len = len(board)
    for i in range(board_len):
        first = board[i][0]
        if first is None:
            continue
        win = True
        for j in range(1, board_len):
            if board[i][j] != first:
                win = False
                break
        if win:
            return first
    return None

In [ ]:
def get_vertical_winner(board):
    """
    Kiểm tra thắng theo cột.
    """
    board_len = len(board)
    for j in range(board_len):
        first = board[0][j]
        if first is None:
            continue
        win = True
        for i in range(1, board_len):
            if board[i][j] != first:
                win = False
                break
        if win:
            return first
    return None

In [ ]:
def get_diagonal_winner(board):
    """
    Kiểm tra thắng theo 2 đường chéo.
    """
    board_len = len(board)

    # Đường chéo chính
    first = board[0][0]
    if first is not None:
        win = True
        for i in range(1, board_len):
            if board[i][i] != first:
                win = False
                break
        if win:
            return first

    # Đường chéo phụ
    first = board[0][board_len - 1]
    if first is not None:
        win = True
        for i in range(1, board_len):
            j = board_len - 1 - i
            if board[i][j] != first:
                win = False
                break
        if win:
            return first

    return None

In [ ]:
def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    return (
        get_horizontal_winner(board)
        or get_vertical_winner(board)
        or get_diagonal_winner(board)
        or None
    )


In [ ]:
def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
    if winner(board) is not None:
        return True

    # Còn ô trống => chưa kết thúc
    for row in board:
        for cell in row:
            if cell == EMPTY:
                return False

    # Hết ô trống => hòa
    return True

In [ ]:
def utility(board):
    """
    Returns 1 if X has won the game,
    -1 if O has won, 0 otherwise.
    """
    w = winner(board)
    if w == X:
        return 1
    elif w == O:
        return -1
    return 0

In [ ]:
def maxValue(state):
    if terminal(state):
        return utility(state)
    v = -math.inf
    for action in actions(state):
        v = max(v, minValue(result(state, action)))
    return v

In [ ]:
def minValue(state):
    if terminal(state):
        return utility(state)
    v = math.inf
    for action in actions(state):
        v = min(v, maxValue(result(state, action)))
    return v

In [ ]:
def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    current_player = player(board)

    # X là MAX, O là MIN
    if current_player == X:
        best_score = -math.inf
        best_move = None
        for action in actions(board):
            score = minValue(result(board, action))
            if score > best_score:
                best_score = score
                best_move = action
    else:
        best_score = math.inf
        best_move = None
        for action in actions(board):
            score = maxValue(result(board, action))
            if score < best_score:
                best_score = score
                best_move = action

    return best_move

In [ ]:
if __name__ == "__main__":
    board = initial_state()
    ai_turn = False

    print("Choose a player (X or O):")
    user = input().strip().upper()
    if user == "X":
        ai = "O"
    else:
        ai = "X"

    while True:
        game_over = terminal(board)
        playr = player(board)

        if game_over:
            w = winner(board)
            if w is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {w} wins.")
            break

        if user != playr and not game_over:
            # lượt AI
            if ai_turn:
                move = minimax(board)
                board = result(board, move)
                ai_turn = False
                print(numpy.array(board))
        elif user == playr and not game_over:
            # lượt người
            ai_turn = True
            print("Enter the position to move (row, col)")
            i = int(input("Row: "))
            j = int(input("Col: "))
            if board[i][j] == EMPTY:
                board = result(board, (i, j))
                print(numpy.array(board))

Choose a player (X or O):
X
Enter the position to move (row, col)
